## LAB 5 [WORKING COPY]

In [3]:
import sklearn
import matplotlib.pyplot as plt
from typing import Tuple
from typing import List
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
%matplotlib inline

import numpy as np
import pandas as pd
import scipy
import pandas as pd 
from sklearn import linear_model
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, classification_report, precision_recall_curve, average_precision_score
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import GradientBoostingClassifier

In [5]:
df_2017 = pd.read_csv("data/2017.csv", low_memory=False, encoding = "ISO-8859-1", nrows=100000, header=None)
# df_2017 = pd.read_csv("../2017.csv", low_memory=False, encoding = "ISO-8859-1", header=None)
df_2017.head()
df_2017.reset_index()

,0,1,2,3,4,5,6,7
0,US1MNWR0029,20170101,PRCP,0,NaN,NaN,N,NaN
1,CA1MB000296,20170101,PRCP,0,NaN,NaN,N,NaN
2,ASN00015643,20170101,TMAX,274,NaN,NaN,a,NaN
3,ASN00015643,20170101,TMIN,218,NaN,NaN,a,NaN
4,ASN00015643,20170101,PRCP,2,NaN,NaN,a,NaN


In [6]:
len(df_2017)

34061507

In [7]:
df_2017.rename(index=int,columns = {0:'ID',1:'date', 2:'element',3:'data_value',4:'mFlag',5:'qFlag',6:'sFlag',7:'obsTime'}, inplace=True)

In [8]:
df_2017.head()

,ID,date,element,data_value,mFlag,qFlag,sFlag,obsTime
0,US1MNWR0029,20170101,PRCP,0,NaN,NaN,N,NaN
1,CA1MB000296,20170101,PRCP,0,NaN,NaN,N,NaN
2,ASN00015643,20170101,TMAX,274,NaN,NaN,a,NaN
3,ASN00015643,20170101,TMIN,218,NaN,NaN,a,NaN
4,ASN00015643,20170101,PRCP,2,NaN,NaN,a,NaN


In [9]:
column_1 = df_2017.ID.values
column_2 = df_2017.date.values


X = np.array(list(zip(column_1,column_2)))
X

array([['US1MNWR0029', '20170101'],
       ['CA1MB000296', '20170101'],
       ['ASN00015643', '20170101'],
       ..., 
       ['ASN00092124', '20171231'],
       ['ASN00092124', '20171231'],
       ['ASN00092124', '20171231']],
      dtype='<U11')

In [10]:
df_2017['merged_index'] = str(df_2017['ID'])+str(df_2017['date'])

In [11]:
df_2017.head()

,ID,date,element,data_value,mFlag,qFlag,sFlag,obsTime,merged_index
0,US1MNWR0029,20170101,PRCP,0,NaN,NaN,N,NaN,0 US1MNWR0029\n1 CA1MB0002...
1,CA1MB000296,20170101,PRCP,0,NaN,NaN,N,NaN,0 US1MNWR0029\n1 CA1MB0002...
2,ASN00015643,20170101,TMAX,274,NaN,NaN,a,NaN,0 US1MNWR0029\n1 CA1MB0002...
3,ASN00015643,20170101,TMIN,218,NaN,NaN,a,NaN,0 US1MNWR0029\n1 CA1MB0002...
4,ASN00015643,20170101,PRCP,2,NaN,NaN,a,NaN,0 US1MNWR0029\n1 CA1MB0002...


In [19]:
df_unstacked = df_2017.pivot( columns='element', values='data_value')

ValueError: negative dimensions are not allowed

In [18]:
df_2017['data_value'].unique()

array([    0,   274,   218, ...,  3910, 26034,  3990], dtype=int64)

In [39]:
frames = [0, 1]
#test = pd.concat([df_2017[[0]], df_2017[[1]]])
#test.head()

df['merged_index'] = pd.merge([df_2017[[0]], df_2017[[1]]], left_index=True)


TypeError: merge() missing 1 required positional argument: 'right'

In [13]:
df_unstacked.head()

NameError: name 'df_unstacked' is not defined

In [25]:
df_2017[2].unique()

array(['PRCP', 'TMAX', 'TMIN', 'SNOW', 'SNWD', 'TAVG', 'WDFG', 'WSFG',
       'WESD', 'WESF', 'AWND', 'WDF2', 'WDF5', 'WSF2', 'WSF5', 'WT01',
       'PGTM', 'WT08', 'TOBS', 'WT02', 'SN52', 'SX52', 'WT06', 'SN32',
       'SX32', 'EVAP', 'MNPN', 'MXPN', 'AWDR', 'WSFI', 'WDMV', 'WT03',
       'SN53', 'SN55', 'SX53', 'SX55', 'DATN', 'MDTN', 'WT09', 'SN31',
       'SN33', 'SX31', 'SX33', 'DAPR', 'MDPR', 'DWPR', 'PSUN', 'TSUN',
       'DATX', 'MDTX', 'THIC', 'WT04', 'SN51', 'SX51', 'WT05', 'WT11',
       'SN35', 'SX35', 'SN36', 'SX36', 'SN56', 'SX56'], dtype=object)

In [59]:
#df_2017['merged'] = df_2017[['ID', 'date']].apply(lambda x: ''.join(str(x)), axis = 0)
df_2017['merged'] = df_2017['ID'].combine_first(df_2017['date'])

In [77]:
df_idd = pd.DataFrame(df_2017[['ID', 'date']])
df_idd.head()

,ID,date
0,US1MNWR0029,20170101
1,CA1MB000296,20170101
2,ASN00015643,20170101
3,ASN00015643,20170101
4,ASN00015643,20170101


In [80]:
df_2017['merged'] = df_2017[['ID','date']].apply(lambda x : '{}{}'.format(x[0],x[1]), axis=1)


In [82]:
df_2017.head()

,ID,date,element,data_value,mFlag,qFlag,sFlag,obsTime,merged_index,merged,period
0,US1MNWR0029,20170101,PRCP,0,NaN,NaN,N,NaN,0 US1MNWR0029\n1 CA1MB000296\n2 ...,US1MNWR002920170101,ID US1MNWR0029\ndate 20170101\nName...
1,CA1MB000296,20170101,PRCP,0,NaN,NaN,N,NaN,0 US1MNWR0029\n1 CA1MB000296\n2 ...,CA1MB00029620170101,ID CA1MB000296\ndate 20170101\nName...
2,ASN00015643,20170101,TMAX,274,NaN,NaN,a,NaN,0 US1MNWR0029\n1 CA1MB000296\n2 ...,ASN0001564320170101,ID ASN00015643\ndate 20170101\nName...
3,ASN00015643,20170101,TMIN,218,NaN,NaN,a,NaN,0 US1MNWR0029\n1 CA1MB000296\n2 ...,ASN0001564320170101,ID ASN00015643\ndate 20170101\nName...
4,ASN00015643,20170101,PRCP,2,NaN,NaN,a,NaN,0 US1MNWR0029\n1 CA1MB000296\n2 ...,ASN0001564320170101,ID ASN00015643\ndate 20170101\nName...


In [83]:
df_unstacked = df_2017.pivot( index='merged', columns='element', values='data_value')

In [84]:
df_unstacked.head()

element,AWDR,AWND,DAPR,DATN,DATX,DWPR,EVAP,MDPR,MDTN,MDTX,...,WSFI,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11
merged,,,,,,,,,,,,,,,,,,,,,
AE00004119620170102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFM0004094820170101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFM0004094820170102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AFM0004099020170102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AG00006039020170102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
df_unstacked_core = df_unstacked[[ 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN']]

In [87]:
df_unstacked_core.head()

element,PRCP,SNOW,SNWD,TMAX,TMIN
merged,,,,,
AE00004119620170102,NaN,NaN,NaN,264.0,NaN
AFM0004094820170101,NaN,NaN,NaN,NaN,NaN
AFM0004094820170102,NaN,NaN,NaN,NaN,NaN
AFM0004099020170102,NaN,NaN,NaN,NaN,NaN
AG00006039020170102,NaN,NaN,NaN,NaN,NaN


In [91]:
df_unstacked_core.isnull().sum()

element
PRCP     2655
SNOW    20071
SNWD    21607
TMAX    20035
TMIN    20050
dtype: int64

In [92]:
len(df_unstacked_core)

32475